# Explore here

In [26]:
# Your code here

# llamo a las librerias necesarias:

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from pickle import dump
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

# lectura, carga y guardado del DF:


# Defino la URL del archivo CSV
url = 'https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv'

# Leer el archivo CSV desde la URL
data_reviews = pd.read_csv(url, sep=',')

# Guardo el DataFrame en un archivo CSV local en la carpeta data con el nombre naive_proyect.csv
data_reviews.to_csv('/workspaces/Bayes-Naive/data/raw/naive_proyect.csv', index=False)

# Análisis de variables:

data_reviews = pd.read_csv('/workspaces/Bayes-Naive/data/raw/naive_proyect.csv', sep=',')
data_reviews.head()

# busco y elimino información no relevante:

data_reviews.drop('package_name', axis=1, inplace=True)

data_reviews

# Elimino espacios y convierto el texto a minúsculas:

def clean_text(review):
    return re.sub(r'[^\w\s]', '', str(review))

data_reviews["review"] = data_reviews["review"].str.strip().str.lower()
data_reviews["review"] = data_reviews["review"].apply(clean_text)

# Separo los datos en entrenamiento y test:

X = data_reviews['review']
y = data_reviews['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creo matriz de recuento de palabras:

vec_model = CountVectorizer(stop_words = "english")
X_train = vec_model.fit_transform(X_train).toarray()
X_test = vec_model.transform(X_test).toarray()

X_train

# Contruir un naive bayes (a elegir entre GaussianNB, MultinomialNB o BernoulliNB)

# eligo el MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred

accuracy_score(y_test, y_pred)

# realizo las otra dos opciones de GaussianNB y BernoulliNB:

for model_aux in [GaussianNB(), BernoulliNB()]:
    model_aux.fit(X_train, y_train)
    y_pred_aux = model_aux.predict(X_test)
    print(f"{model_aux} with accuracy: {accuracy_score(y_test, y_pred_aux)}")


# La mejor opcion es la Multinomial.

# Optimizamos el modelo:

hyperparams = {
    "alpha": np.linspace(0.01, 10.0, 20),
    "force_alpha": [True,False]
}

random_search = RandomizedSearchCV(model, hyperparams, n_iter = 50, scoring = "accuracy", cv = 5, random_state = 42)
random_search

random_search.fit(X_train, y_train)
random_search.best_estimator_

model_MN = MultinomialNB(**random_search.best_params_)

# Entrenamiento y evaluacion de la predicción:

model_MN.fit(X_train, y_train)
y_pred_2 = model.predict(X_test)
print(model_MN.get_params())
print("Accuracy NM:", accuracy_score(y_test, y_pred_2))


# Guardo el modelo en la carpeta models:

dump(model, open("/workspaces/Bayes-Naive/models/naive_proyect_optimizado.sav", "wb"))


# Explorando otras altenativas:

# árbol de decisión (DT):

DT = DecisionTreeClassifier(random_state = 42)
DT.fit(X_train, y_train)
y_pred_3 = DT.predict(X_test)
y_pred_train = DT.predict(X_train)
print('Accuracy train DT: ', accuracy_score(y_pred_train, y_train))
print('Accuracy test DT: ', accuracy_score(y_pred_3, y_test))

# Regresión Logística (RL):

RL = LogisticRegression()
RL.fit(X_train, y_train)
y_pred_4 = RL.predict(X_test)
y_pred_train = RL.predict(X_train)
print('Accuracy train RL: ', accuracy_score(y_pred_train, y_train))
print('Accuracy test RL: ', accuracy_score(y_pred_4, y_test))

# Random Forest (RF):

RF = RandomForestClassifier(random_state = 42)
RF.fit(X_train, y_train)
y_pred_5 = RF.predict(X_test)
y_pred_train = RF.predict(X_train)
print('Accuracy train RF: ', accuracy_score(y_pred_train, y_train))
print('Accuracy test RF: ', accuracy_score(y_pred_5, y_test))




GaussianNB() with accuracy: 0.7988826815642458
BernoulliNB() with accuracy: 0.770949720670391


/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 40 is smaller than n_iter=50. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'alpha': 1.5873684210526318, 'class_prior': None, 'fit_prior': True, 'force_alpha': True}
Accuracy NM: 0.7988826815642458
Accuracy train DT:  1.0
Accuracy test DT:  0.7430167597765364
Accuracy train RL:  0.9985955056179775
Accuracy test RL:  0.8212290502793296
Accuracy train RF:  1.0
Accuracy test RF:  0.8044692737430168


El modelo de Regresión logistica como alternativa, es el que mejor análisis arroja sobre el problema.